## Forecasting Honda Civic Sales

In [1]:
# Import packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

%matplotlib inline

In [2]:
# Define data frame
data = pd.read_csv("Civic-142A-Fall25.csv")
data.head()

,MonthNumeric,MonthFactor,Year,CivicSales,Unemployment,CivicQueries,CPIAll,CPIEnergy,MilesTraveled
0,1,January,2014,21824,6.6,66,235.288,250.340,246531
1,2,February,2014,21575,6.7,69,235.547,249.925,249499
2,3,March,2014,27697,6.7,72,236.028,249.961,251120
3,4,April,2014,27611,6.2,69,236.468,249.864,251959
4,5,May,2014,36089,6.3,69,236.918,249.213,252289


In [3]:
# Split data into tranning set and testing set
data_train = data[data['Year'] <= 2019]
data_test = data[data['Year'] >= 2020]

# Train model base on log scale
data_train = data_train.copy()

In [4]:
# Choose the features to be used
cols5 = ['Unemployment','CivicQueries','CPIEnergy','CPIAll','MilesTraveled']
X_train = data_train[cols5]
y_train = data_train[['CivicSales']]

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)
X_train.head()

,const,Unemployment,CivicQueries,CPIEnergy,CPIAll,MilesTraveled
0,1.0,6.6,66,250.340,235.288,246531
1,1.0,6.7,69,249.925,235.547,249499
2,1.0,6.7,72,249.961,236.028,251120
3,1.0,6.2,69,249.864,236.468,251959
4,1.0,6.3,69,249.213,236.918,252289


#### Evaluating the model

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def VIF(df, columns):
# The dataframe passed to VIF must include the intercept term. We add itthe same way we did before.
    
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    
    return pd.Series(vif[1:], index=columns)

In [6]:
# Compute out-of-sample R-squared using the test set
def OSR2(model, df_train, df_test, dependent_var):
    
    y_test = df_test[dependent_var]
    y_pred = model.predict(df_test)
    
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(df_train[dependent_var]))**2)
    
    return 1 - SSE/SST

### Training the model

In [7]:
# cols5 = ['Unemployment','CivicQueries','CPIEnergy','CPIAll','MilesTraveled']
X_train = data_train[cols5]

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)
X_train

model1 = sm.OLS(y_train, X_train).fit() #ordinary least square
print(model1.summary())

VIF(data_train, cols5)

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     9.898
Date:                Thu, 25 Sep 2025   Prob (F-statistic):           4.29e-07
Time:                        20:42:38   Log-Likelihood:                -687.82
No. Observations:                  72   AIC:                             1388.
Df Residuals:                      66   BIC:                             1401.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          1.298e+04   7.37e+04      0.176

Unemployment     24.600912
CivicQueries      1.717734
CPIEnergy         5.033046
CPIAll           23.929349
MilesTraveled    12.647068
dtype: float64

In [8]:
# Remove the Unemployment feature because of the highest VIF (24.6)
# And keep everything the same

cols2 = ['CivicQueries','CPIEnergy','CPIAll','MilesTraveled']
X_train = data_train[cols2]

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)
X_train

model2 = sm.OLS(y_train, X_train).fit() #ordinary least square
print(model2.summary())

VIF(data_train, cols2)

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.428
Model:                            OLS   Adj. R-squared:                  0.394
Method:                 Least Squares   F-statistic:                     12.55
Date:                Thu, 25 Sep 2025   Prob (F-statistic):           1.13e-07
Time:                        20:42:38   Log-Likelihood:                -687.84
No. Observations:                  72   AIC:                             1386.
Df Residuals:                      67   BIC:                             1397.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2.565e+04   2.22e+04      1.155

CivicQueries      1.683027
CPIEnergy         2.962402
CPIAll            9.720030
MilesTraveled    11.995681
dtype: float64

In [9]:
# Remove the MilesTraveled feature because of the highest VIF (12.00)
# And keep everything the same

cols3 = ['CivicQueries', 'CPIEnergy', 'CPIAll']
X_train = data_train[cols3]

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)
X_train

model3 = sm.OLS(y_train, X_train).fit() #ordinary least square
print(model3.summary())
VIF(data_train, cols3)

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.378
Method:                 Least Squares   F-statistic:                     15.40
Date:                Thu, 25 Sep 2025   Prob (F-statistic):           9.51e-08
Time:                        20:42:38   Log-Likelihood:                -689.29
No. Observations:                  72   AIC:                             1387.
Df Residuals:                      68   BIC:                             1396.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         5.338e+04   1.49e+04      3.589   

CivicQueries    1.578403
CPIEnergy       1.290031
CPIAll          1.318700
dtype: float64

In [10]:
# Remove the CPIEnergy feature because of the high p-value (0.295)
# And keep everything the same

cols4 = ['CivicQueries', 'CPIAll']
X_train = data_train[cols4]

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)
X_train

model4 = sm.OLS(y_train, X_train).fit() #ordinary least square
print(model4.summary())
VIF(data_train, cols4)

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.395
Model:                            OLS   Adj. R-squared:                  0.377
Method:                 Least Squares   F-statistic:                     22.51
Date:                Thu, 25 Sep 2025   Prob (F-statistic):           2.98e-08
Time:                        20:42:38   Log-Likelihood:                -689.88
No. Observations:                  72   AIC:                             1386.
Df Residuals:                      69   BIC:                             1393.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         5.713e+04   1.45e+04      3.953   

CivicQueries    1.225893
CPIAll          1.225893
dtype: float64

In [11]:
# Train predictions
y_train_pred = model4.predict()

# Metrics
rmse_train = np.sqrt(mean_squared_error(data_train['CivicSales'], y_train_pred))
mae_train = mean_absolute_error(data_train['CivicSales'], y_train_pred)
r2_train = r2_score(data_train['CivicSales'], y_train_pred)

print(f"\nTrain RMSE: {rmse_train:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Train R²: {r2_train:.4f}")


Train RMSE: 3507.71
Train MAE: 2839.92
Train R²: 0.3949


Comment:

- R² = 0.395: Explains ~39.5% of the variance in monthly Civic sales.

- RMSE = 3,508 cars: Average prediction error is about 3,500 cars per month.

- MAE = 2,840 cars: Typical prediction is within ~2,800 cars of actual sales.

The full five-variable model had a slightly higher R² (0.429), but it suffered from severe multicollinearity and included non-significant predictors. This final model achieves nearly equivalent explanatory power (Adj. R-squared: 3.77 vs 3.95) with only two robust predictors, making it more reliable and interpretable.

\newpage

In [12]:
formula_b = "CivicSales ~ Unemployment + CivicQueries + CPIEnergy + CPIAll + MilesTraveled + C(MonthFactor)"
model_b = smf.ols(formula=formula_b, data=data_train).fit()
print(model_b.summary())

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.697
Method:                 Least Squares   F-statistic:                     11.19
Date:                Thu, 25 Sep 2025   Prob (F-statistic):           5.05e-12
Time:                        20:42:38   Log-Likelihood:                -655.83
No. Observations:                  72   AIC:                             1346.
Df Residuals:                      55   BIC:                             1384.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [13]:
y_train_pred = model_b.predict(data_train)

rmse_train_b = np.sqrt(mean_squared_error(data_train['CivicSales'],y_train_pred))
mae_train_b = mean_absolute_error(data_train['CivicSales'], y_train_pred)
OSR2B = OSR2(model_b, data_train, data_test, 'CivicSales')

print(f"\nTrain RMSE: {rmse_train_b:.2f}")
print(f"Train MAE: {mae_train_b:.2f}")
print(f"Train R²: {model_b.rsquared:.4f}")
print(f"OSR² {OSR2B:.4f}")


Train RMSE: 2185.89
Train MAE: 1667.35
Train R²: 0.7650
OSR² 0.8294


/newpage

In [14]:
# only Month 1, 2, 5 had p-value < 0.05
# selected month 1,2,4,5 for the data set
data_c = data[data['MonthNumeric'].isin([1, 2, 5, 4])]
data_train_c = data_c[data_c['Year'] <= 2019]
data_test_c = data_c[data_c['Year'] >= 2020]

In [15]:
formula_c = "CivicSales ~ CivicQueries + CPIAll  + C(MonthFactor)"
model_c = smf.ols(formula=formula_c, data=data_train_c).fit()
print(model_c.summary())

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.910
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     36.42
Date:                Thu, 25 Sep 2025   Prob (F-statistic):           8.39e-09
Time:                        20:42:38   Log-Likelihood:                -210.25
No. Observations:                  24   AIC:                             432.5
Df Residuals:                      18   BIC:                             439.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
y_train_c = model_c.predict(data_train_c)

rmse_train_c = np.sqrt(mean_squared_error(data_train_c['CivicSales'],y_train_c))
mae_train_c = mean_absolute_error(data_train_c['CivicSales'], y_train_c)
OSR2C = OSR2(model_c, data_train_c, data_test_c, 'CivicSales')

print(f"\nTrain RMSE: {rmse_train_c:.2f}")
print(f"Train MAE: {mae_train_c:.2f}")
print(f"Train R²: {model_c.rsquared:.4f}")
print(f"OSR² {OSR2C:.4f}")


Train RMSE: 1543.13
Train MAE: 1097.00
Train R²: 0.9100
OSR² 0.6434


- At this point I think this model might overfit. The gap between $R^2$ (0.91) and $OSR^2$ (0.64) is expected: in-sample fit is higher, while out-of-sample performance reflects 2020+ shifts and less data per month. Even so, $OSR^2$ (0.64)4 indicates good generalization on the selected months.


In [17]:
#Traning Set for R^2
data_train_c.head()

,MonthNumeric,MonthFactor,Year,CivicSales,Unemployment,CivicQueries,CPIAll,CPIEnergy,MilesTraveled
0,1,January,2014,21824,6.6,66,235.288,250.340,246531
1,2,February,2014,21575,6.7,69,235.547,249.925,249499
3,4,April,2014,27611,6.2,69,236.468,249.864,251959
4,5,May,2014,36089,6.3,69,236.918,249.213,252289
12,1,January,2015,18699,5.7,58,234.747,199.926,254967


### Add new feature

- I'm adding new feature name AutoLoanRate for the model using the the data from FRED page
- I think this will have significant impact on this model because the AutoLoanRate will affect the impact of customer who want to buy car.

In [18]:
# Load and prepare auto loan data
auto_loan_data = pd.read_csv('AutoLoanRate.csv')

# Convert observation_date to datetime and extract year and month
auto_loan_data['observation_date'] = pd.to_datetime(auto_loan_data['observation_date'])
auto_loan_data['Year'] = auto_loan_data['observation_date'].dt.year
auto_loan_data['MonthNumeric'] = auto_loan_data['observation_date'].dt.month  # Use MonthNumeric to match your main data

# Handle missing values - forward fill then backward fill
auto_loan_data['TERMCBAUTO48NS'] = auto_loan_data['TERMCBAUTO48NS'].replace('', np.nan)
auto_loan_data['AutoLoanRate'] = auto_loan_data['TERMCBAUTO48NS'].astype(float)
auto_loan_data['AutoLoanRate'] = auto_loan_data['AutoLoanRate'].ffill().bfill()

# Keep only necessary columns and drop duplicates
auto_loan_clean = auto_loan_data[['Year', 'MonthNumeric', 'AutoLoanRate']].drop_duplicates()

# Merge 
data_d = data_c.merge(auto_loan_clean, on=['Year', 'MonthNumeric'], how='left')

In [19]:
data_train_d = data_d[data_d['Year'] <= 2019]
data_test_d = data_d[data_d['Year'] >= 2020]

In [20]:
formula_d = "CivicSales ~ CivicQueries + CPIAll + AutoLoanRate + C(MonthFactor)"
model_d = smf.ols(formula=formula_d, data=data_train_d).fit()
print(model_d.summary())

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.913
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     29.86
Date:                Thu, 25 Sep 2025   Prob (F-statistic):           3.97e-08
Time:                        20:42:38   Log-Likelihood:                -209.80
No. Observations:                  24   AIC:                             433.6
Df Residuals:                      17   BIC:                             441.9
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [21]:
y_train_d = model_d.predict(data_train_d)

rmse_train_d = np.sqrt(mean_squared_error(data_train_d['CivicSales'],y_train_d))
mae_train_d = mean_absolute_error(data_train_d['CivicSales'], y_train_d)
OSR2D = OSR2(model_d, data_train_d, data_test_d, 'CivicSales')

print(f"\nTrain RMSE: {rmse_train_d:.2f}")
print(f"Train MAE: {mae_train_d:.2f}")
print(f"Train R²: {model_d.rsquared:.4f}")
print(f"OSR² {OSR2D:.4f}")


Train RMSE: 1514.56
Train MAE: 1056.41
Train R²: 0.9133
OSR² 0.4809


### Conclusion:
The new variable have impact on the model where the $R^2$ to 0.9133 but the $OSR^2$ is drop rapiddly. This feature also make the CPIAll feature p-value become higher (0.776). Base on this, I think this feature just make this model more overfit.